<h2>Parsing & Cleaning JSON Data</h2>
<h4>This is a short walkthrough of how to clean JSON data, which can be very tricky because fields can be embedded or formatted incorrectly.</h4>

In [1]:
import pandas as pd
import numpy as np
import urllib
import re
import ast
import json

<h3>Step 1: Downloading the dataset</h3>
<p>We are working with data that contains information on nobel prize winners. The data comes from an open dataset in an API format. Authentication is not required so the dataset can be freely downloaded wthout a key.</p>
<p>The first problem we run into is that double quotes are utilized. If we attempt to parse the data without getting rid of these we will run into errors. To circumvent this, we will dump the data and replace the double quotes with blanks. This way when the data can be parsed as a JSON dataset correctly using ast.literal_eval</p>

In [2]:
url = "http://api.nobelprize.org/v1/prize.json"
data= json.load(urllib.request.urlopen(url))
data2 = json.dumps(data)

In [3]:
data2 = data2.replace('\\"','')

In [4]:
data2

'{"prizes": [{"year": "2019", "category": "chemistry", "laureates": [{"id": "976", "firstname": "John", "surname": "Goodenough", "motivation": "for the development of lithium-ion batteries", "share": "3"}, {"id": "977", "firstname": "M. Stanley", "surname": "Whittingham", "motivation": "for the development of lithium-ion batteries", "share": "3"}, {"id": "978", "firstname": "Akira", "surname": "Yoshino", "motivation": "for the development of lithium-ion batteries", "share": "3"}]}, {"year": "2019", "category": "economics", "laureates": [{"id": "982", "firstname": "Abhijit", "surname": "Banerjee", "motivation": "for their experimental approach to alleviating global poverty", "share": "3"}, {"id": "983", "firstname": "Esther", "surname": "Duflo", "motivation": "for their experimental approach to alleviating global poverty", "share": "3"}, {"id": "984", "firstname": "Michael", "surname": "Kremer", "motivation": "for their experimental approach to alleviating global poverty", "share": "3"}

In [5]:
data2 = ast.literal_eval(data2)

In [6]:
data2

{'prizes': [{'year': '2019',
   'category': 'chemistry',
   'laureates': [{'id': '976',
     'firstname': 'John',
     'surname': 'Goodenough',
     'motivation': 'for the development of lithium-ion batteries',
     'share': '3'},
    {'id': '977',
     'firstname': 'M. Stanley',
     'surname': 'Whittingham',
     'motivation': 'for the development of lithium-ion batteries',
     'share': '3'},
    {'id': '978',
     'firstname': 'Akira',
     'surname': 'Yoshino',
     'motivation': 'for the development of lithium-ion batteries',
     'share': '3'}]},
  {'year': '2019',
   'category': 'economics',
   'laureates': [{'id': '982',
     'firstname': 'Abhijit',
     'surname': 'Banerjee',
     'motivation': 'for their experimental approach to alleviating global poverty',
     'share': '3'},
    {'id': '983',
     'firstname': 'Esther',
     'surname': 'Duflo',
     'motivation': 'for their experimental approach to alleviating global poverty',
     'share': '3'},
    {'id': '984',
     'fi

<h3>Step 2: Flattening the embedded data</h3> 
<p>Now that we have the data in a form that can be correclty parsed we need to turn the embedded data into a flat dataset.</p>

In [7]:
df = pd.json_normalize(data2['prizes'])

In [8]:
df.head()

,year,category,laureates,overallMotivation
0,2019,chemistry,"[{'id': '976', 'firstname': 'John', 'surname':...",NaN
1,2019,economics,"[{'id': '982', 'firstname': 'Abhijit', 'surnam...",NaN
2,2019,literature,"[{'id': '980', 'firstname': 'Peter', 'surname'...",NaN
3,2019,peace,"[{'id': '981', 'firstname': 'Abiy', 'surname':...",NaN
4,2019,physics,"[{'id': '973', 'firstname': 'James', 'surname'...",for contributions to our understanding of the ...


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646 entries, 0 to 645
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   year               646 non-null    object
 1   category           646 non-null    object
 2   laureates          597 non-null    object
 3   overallMotivation  57 non-null     object
dtypes: object(4)
memory usage: 20.3+ KB


In [10]:
df['laureates'][0]

[{'id': '976',
  'firstname': 'John',
  'surname': 'Goodenough',
  'motivation': 'for the development of lithium-ion batteries',
  'share': '3'},
 {'id': '977',
  'firstname': 'M. Stanley',
  'surname': 'Whittingham',
  'motivation': 'for the development of lithium-ion batteries',
  'share': '3'},
 {'id': '978',
  'firstname': 'Akira',
  'surname': 'Yoshino',
  'motivation': 'for the development of lithium-ion batteries',
  'share': '3'}]

<p>There appears to be an "id" field embedded in the laureates column as well as info about the nobel prize winners when more than one individual wins the prize. We will  break the data out and have one user on each row.

<h4>A row for each winner</h4>

In [11]:
laureates = (df.laureates.apply(pd.Series).stack().reset_index(level=1, drop=True).to_frame('laureates'))

In [12]:
laureates.head()

,laureates
0,"{'id': '976', 'firstname': 'John', 'surname': ..."
0,"{'id': '977', 'firstname': 'M. Stanley', 'surn..."
0,"{'id': '978', 'firstname': 'Akira', 'surname':..."
1,"{'id': '982', 'firstname': 'Abhijit', 'surname..."
1,"{'id': '983', 'firstname': 'Esther', 'surname'..."


<p>We need to add a key so that we can join the dataframes together again</p>

In [13]:
df['key'] = df.index
laureates['key'] = laureates.index

In [14]:
df = df.join(laureates, on='key', how='left', lsuffix='_left', rsuffix='right')

In [15]:
df.drop(columns=['laureates_left'], inplace=True)

In [16]:
df['laureatesright'].head()

0    {'id': '976', 'firstname': 'John', 'surname': ...
0    {'id': '977', 'firstname': 'M. Stanley', 'surn...
0    {'id': '978', 'firstname': 'Akira', 'surname':...
1    {'id': '982', 'firstname': 'Abhijit', 'surname...
1    {'id': '983', 'firstname': 'Esther', 'surname'...
Name: laureatesright, dtype: object

In [17]:
df = df.reset_index(drop=True)

In [18]:
df.head()

,key,year,category,overallMotivation,key_left,laureatesright,keyright
0,0,2019,chemistry,NaN,0,"{'id': '976', 'firstname': 'John', 'surname': ...",0.0
1,0,2019,chemistry,NaN,0,"{'id': '977', 'firstname': 'M. Stanley', 'surn...",0.0
2,0,2019,chemistry,NaN,0,"{'id': '978', 'firstname': 'Akira', 'surname':...",0.0
3,1,2019,economics,NaN,1,"{'id': '982', 'firstname': 'Abhijit', 'surname...",1.0
4,1,2019,economics,NaN,1,"{'id': '983', 'firstname': 'Esther', 'surname'...",1.0


<p>We now have a row for every winner and the only other step we need to make is creating individual columns for the data in the laureates column.</p>

In [19]:
import requests
import codecs
import json

from codecs import raw_unicode_escape_decode

In [20]:
df3 = df['laureatesright'].apply(pd.Series).reset_index(drop=True)
df = df.reset_index(drop=True)

In [21]:
df['key'] = df.index
df3['key'] = df3.index

In [22]:
df = df.merge(df3, left_on='key', right_on='key')

<p>We've done it. We now have a column for each of the nobel laureates</p>

<h3>Step 3: Cleaning the Data</h3>
<p>Now that we have a flat file we will clean the dataset</p>

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 999 entries, 0 to 998
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   key                999 non-null    int64  
 1   year               999 non-null    object 
 2   category           999 non-null    object 
 3   overallMotivation  69 non-null     object 
 4   key_left           999 non-null    int64  
 5   laureatesright     950 non-null    object 
 6   keyright           950 non-null    float64
 7   0                  0 non-null      float64
 8   firstname          950 non-null    object 
 9   id                 950 non-null    object 
 10  motivation         950 non-null    object 
 11  share              950 non-null    object 
 12  surname            921 non-null    object 
dtypes: float64(2), int64(2), object(9)
memory usage: 109.3+ KB


<p>There appear to be 49 rows where no winner is listed. We'll find out if this is an issue with the data or if there is an explanation.</p>

In [24]:
df_nan = df[df[['laureatesright']].isna().any(axis=1)]

In [25]:
df_nan.head()

,key,year,category,overallMotivation,key_left,laureatesright,keyright,0,firstname,id,motivation,share,surname
529,529,1972,peace,No Nobel Prize was awarded this year. The priz...,285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
571,571,1967,peace,No Nobel Prize was awarded this year. The priz...,313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
579,579,1966,peace,No Nobel Prize was awarded this year. The priz...,318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
656,656,1956,peace,No Nobel Prize was awarded this year. The priz...,368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
665,665,1955,peace,No Nobel Prize was awarded this year. The priz...,373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df_nan['overallMotivation'].value_counts()

No Nobel Prize was awarded this year. The prize money was allocated to the Special Fund of this prize section.                                           24
No Nobel Prize was awarded this year. The prize money was with 1/3 allocated to the Main Fund and with 2/3 to the Special Fund of this prize section.    24
No Nobel Prize was awarded this year. The prize money for 1972 was allocated to the Main Fund.                                                            1
Name: overallMotivation, dtype: int64

<p>We see that in the cases where there were no winners in the rows where the laureate column is null. We can delete these rows. Also, we no longer need the key columns</p>

In [27]:
df = df[df['laureatesright'].notna()]

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 950 entries, 0 to 998
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   key                950 non-null    int64  
 1   year               950 non-null    object 
 2   category           950 non-null    object 
 3   overallMotivation  20 non-null     object 
 4   key_left           950 non-null    int64  
 5   laureatesright     950 non-null    object 
 6   keyright           950 non-null    float64
 7   0                  0 non-null      float64
 8   firstname          950 non-null    object 
 9   id                 950 non-null    object 
 10  motivation         950 non-null    object 
 11  share              950 non-null    object 
 12  surname            921 non-null    object 
dtypes: float64(2), int64(2), object(9)
memory usage: 103.9+ KB


<h4>Getting rid of unneeded columns/reformatting columns</h4>
We can now get rid of redundant columns, columns used for re-merging the data and reformat columns.

In [29]:
df = df[['year', 'category', 'overallMotivation', 'id','firstname','surname', 'motivation', 'share']]

In [30]:
df.head()

,year,category,overallMotivation,id,firstname,surname,motivation,share
0,2019,chemistry,NaN,976,John,Goodenough,for the development of lithium-ion batteries,3
1,2019,chemistry,NaN,977,M. Stanley,Whittingham,for the development of lithium-ion batteries,3
2,2019,chemistry,NaN,978,Akira,Yoshino,for the development of lithium-ion batteries,3
3,2019,economics,NaN,982,Abhijit,Banerjee,for their experimental approach to alleviating...,3
4,2019,economics,NaN,983,Esther,Duflo,for their experimental approach to alleviating...,3


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 950 entries, 0 to 998
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   year               950 non-null    object
 1   category           950 non-null    object
 2   overallMotivation  20 non-null     object
 3   id                 950 non-null    object
 4   firstname          950 non-null    object
 5   surname            921 non-null    object
 6   motivation         950 non-null    object
 7   share              950 non-null    object
dtypes: object(8)
memory usage: 66.8+ KB


<p>We should now cast our numerical column as integers</p>

In [32]:
df.index =df.id.astype(int)

In [39]:
df['year'] = df['year'].astype(int)
df['share'] = df['share'].astype(int)

In [33]:
df = df[['year', 'category', 'overallMotivation', 'firstname','surname', 'motivation', 'share']]

In [34]:
df

,year,category,overallMotivation,firstname,surname,motivation,share
id,,,,,,,
976,2019,chemistry,NaN,John,Goodenough,for the development of lithium-ion batteries,3
977,2019,chemistry,NaN,M. Stanley,Whittingham,for the development of lithium-ion batteries,3
978,2019,chemistry,NaN,Akira,Yoshino,for the development of lithium-ion batteries,3
982,2019,economics,NaN,Abhijit,Banerjee,for their experimental approach to alleviating...,3
983,2019,economics,NaN,Esther,Duflo,for their experimental approach to alleviating...,3
...,...,...,...,...,...,...,...
569,1901,literature,NaN,Sully,Prudhomme,in special recognition of his poetic compositi...,1
462,1901,peace,NaN,Henry,Dunant,for his humanitarian efforts to help wounded s...,2
463,1901,peace,NaN,Frédéric,Passy,for his lifelong work for international peace ...,2


<h4>Addressing sparse columns</h4>

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 950 entries, 976 to 293
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   year               950 non-null    object
 1   category           950 non-null    object
 2   overallMotivation  20 non-null     object
 3   firstname          950 non-null    object
 4   surname            921 non-null    object
 5   motivation         950 non-null    object
 6   share              950 non-null    object
dtypes: object(7)
memory usage: 59.4+ KB


In [36]:
df.overallMotivation.value_counts(dropna=False)

NaN                                                                                                                 930
for the development of methods for identification and structure analyses of biological macromolecules                 3
for contributions to our understanding of the evolution of the universe and Earth’s place in the cosmos               3
for groundbreaking inventions in the field of laser physics                                                           3
for basic work on information and communication technology                                                            3
for discoveries concerning channels in cell membranes                                                                 2
for pioneering contributions to the development of neutron scattering techniques for studies of condensed matter      2
for contributions to the developments of methods within DNA-based chemistry                                           2
for pioneering experimental contribution

<p>We see that motivation is split up across two columns. In some cases it appears that when more than one person shared the award they may have had different motivations. We will concatenate the two fields.</p>

In [37]:
df[df['overallMotivation'].notna()]

,year,category,overallMotivation,firstname,surname,motivation,share
id,,,,,,,
973,2019,physics,for contributions to our understanding of the ...,James,Peebles,for theoretical discoveries in physical cosmology,2
974,2019,physics,for contributions to our understanding of the ...,Michel,Mayor,for the discovery of an exoplanet orbiting a s...,4
975,2019,physics,for contributions to our understanding of the ...,Didier,Queloz,for the discovery of an exoplanet orbiting a s...,4
960,2018,physics,for groundbreaking inventions in the field of ...,Arthur,Ashkin,for the optical tweezers and their application...,2
961,2018,physics,for groundbreaking inventions in the field of ...,Gérard,Mourou,"for their method of generating high-intensity,...",4
962,2018,physics,for groundbreaking inventions in the field of ...,Donna,Strickland,"for their method of generating high-intensity,...",4
769,2003,chemistry,for discoveries concerning channels in cell me...,Peter,Agre,for the discovery of water channels,2
770,2003,chemistry,for discoveries concerning channels in cell me...,Roderick,MacKinnon,for structural and mechanistic studies of ion ...,2
756,2002,chemistry,for the development of methods for identificat...,John B.,Fenn,for their development of soft desorption ionis...,4


In [38]:
df['motivation'] = df['overallMotivation'].fillna('') +  df['motivation'] 

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 950 entries, 976 to 293
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   year               950 non-null    int64 
 1   category           950 non-null    object
 2   overallMotivation  20 non-null     object
 3   firstname          950 non-null    object
 4   surname            921 non-null    object
 5   motivation         950 non-null    object
 6   share              950 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 59.4+ KB


In [41]:
df.drop(columns=['overallMotivation'], inplace=True)

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 950 entries, 976 to 293
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   year        950 non-null    int64 
 1   category    950 non-null    object
 2   firstname   950 non-null    object
 3   surname     921 non-null    object
 4   motivation  950 non-null    object
 5   share       950 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 52.0+ KB


In [43]:
df = df.sort_index()

In [44]:
df.head()

,year,category,firstname,surname,motivation,share
id,,,,,,
1,1901,physics,Wilhelm Conrad,Röntgen,in recognition of the extraordinary services h...,1
2,1902,physics,Hendrik A.,Lorentz,in recognition of the extraordinary service th...,2
3,1902,physics,Pieter,Zeeman,in recognition of the extraordinary service th...,2
4,1903,physics,Henri,Becquerel,in recognition of the extraordinary services h...,2
5,1903,physics,Pierre,Curie,in recognition of the extraordinary services t...,4


In [45]:
df.tail()

,year,category,firstname,surname,motivation,share
id,,,,,,
980,2019,literature,Peter,Handke,for an influential work that with linguistic i...,1
981,2019,peace,Abiy,Ahmed Ali,for his efforts to achieve peace and internati...,1
982,2019,economics,Abhijit,Banerjee,for their experimental approach to alleviating...,3
983,2019,economics,Esther,Duflo,for their experimental approach to alleviating...,3
984,2019,economics,Michael,Kremer,for their experimental approach to alleviating...,3


<h3>Conclusion</h3>
<h4>We now have a succinct data set of nobel laureates, properly formatted and cleaned.</h4>